# 量子アニーリングによる3Dパッキング最適化 (Fixstars Amplify FMQA)

本ノートブックでは、Fixstars Amplify の FMQA（Factorization Machineによるブラックボックス最適化）を用いて、
60個の段ボールを1つの輸送箱に効率良く詰める問題を解きます。

## 🔍 問題概要

- 段ボールのサイズは8種類、各段ボールは90度回転可（荷姿2通り）
- 評価軸：
  - 充填率（体積効率）
  - 重心位置（箱の中心±10%以内が望ましい）
  - 安定性（横方向の面接触：最大4面）


In [ ]:
!pip install amplify --quiet

In [ ]:

import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
from amplify import BinarySymbol, FixstarsClient, FMQASampler


## 📦 荷姿候補の生成

In [ ]:

BOX_SIZE = (100, 100, 100)
CARTON_TYPES = [(20, 30, 10), (40, 20, 10), (10, 10, 10),
                (30, 30, 20), (50, 40, 20), (20, 20, 20),
                (60, 30, 15), (25, 25, 25)]

NUM_CANDIDATES = 100
GRID_UNIT = 10
NUM_CARTONS = 60

np.random.seed(42)
CARTON_ASSIGNMENTS = np.random.choice(len(CARTON_TYPES), size=NUM_CARTONS)

def generate_placement_candidates(carton_type_id):
    w, d, h = CARTON_TYPES[carton_type_id]
    poses = [(w, d, h), (d, w, h)]
    candidates = []
    for pose in poses:
        pw, pd, ph = pose
        for x in range(0, BOX_SIZE[0] - pw + 1, GRID_UNIT):
            for y in range(0, BOX_SIZE[1] - pd + 1, GRID_UNIT):
                for z in range(0, BOX_SIZE[2] - ph + 1, GRID_UNIT):
                    candidates.append({
                        'position': (x, y, z),
                        'size': (pw, pd, ph)
                    })
    np.random.shuffle(candidates)
    return candidates[:NUM_CANDIDATES]

carton_placement_candidates = [
    generate_placement_candidates(ct_id) for ct_id in CARTON_ASSIGNMENTS
]


## ⚠️ 衝突判定関数

In [ ]:

def check_collision(pos1, size1, pos2, size2):
    for i in range(3):
        if pos1[i] + size1[i] <= pos2[i] or pos2[i] + size2[i] <= pos1[i]:
            continue
        else:
            return True
    return False


## 🧠 評価関数の定義

In [ ]:

def evaluate_solution(assignments):
    used_volume = 0
    total_volume = BOX_SIZE[0] * BOX_SIZE[1] * BOX_SIZE[2]
    positions, sizes, mass_centers = [], [], []

    for i, candidate_id in enumerate(assignments):
        candidate = carton_placement_candidates[i][candidate_id]
        pos = candidate['position']
        size = candidate['size']
        positions.append(pos)
        sizes.append(size)
        used_volume += size[0] * size[1] * size[2]
        mass_centers.append([pos[j] + size[j] / 2 for j in range(3)])

    for i in range(NUM_CARTONS):
        for j in range(i + 1, NUM_CARTONS):
            if check_collision(positions[i], sizes[i], positions[j], sizes[j]):
                return -1e6

    packing_efficiency = used_volume / total_volume
    gravity_center = np.mean(mass_centers, axis=0)
    target = np.array(BOX_SIZE) / 2
    gravity_penalty = np.linalg.norm(gravity_center - target) / np.linalg.norm(target)

    stability_score = 0
    for i in range(NUM_CARTONS):
        pos1, size1 = positions[i], sizes[i]
        neighbors = 0
        for j in range(NUM_CARTONS):
            if i == j:
                continue
            pos2, size2 = positions[j], sizes[j]
            for dim in range(2):
                if abs(pos1[dim] + size1[dim] - pos2[dim]) <= 1 or abs(pos2[dim] + size2[dim] - pos1[dim]) <= 1:
                    if abs(pos1[2] - pos2[2]) < GRID_UNIT:
                        neighbors += 1
        stability_score += min(neighbors, 4)
    stability_score = stability_score / (4 * NUM_CARTONS)

    return packing_efficiency - gravity_penalty + stability_score


## 📊 可視化関数（3D）

In [ ]:

def visualize_packing(assignments):
    fig = plt.figure(figsize=(10, 8))
    ax = fig.add_subplot(111, projection='3d')

    def draw_box(pos, size, color):
        x, y, z = pos
        dx, dy, dz = size
        vertices = [
            [x, y, z], [x+dx, y, z], [x+dx, y+dy, z], [x, y+dy, z],
            [x, y, z+dz], [x+dx, y, z+dz], [x+dx, y+dy, z+dz], [x, y+dy, z+dz]
        ]
        faces = [
            [vertices[i] for i in [0,1,2,3]],
            [vertices[i] for i in [4,5,6,7]],
            [vertices[i] for i in [0,1,5,4]],
            [vertices[i] for i in [2,3,7,6]],
            [vertices[i] for i in [1,2,6,5]],
            [vertices[i] for i in [0,3,7,4]],
        ]
        box = Poly3DCollection(faces, alpha=0.6)
        box.set_facecolor(color)
        ax.add_collection3d(box)

    for i, candidate_id in enumerate(assignments):
        candidate = carton_placement_candidates[i][candidate_id]
        color = np.random.rand(3,)
        draw_box(candidate['position'], candidate['size'], color)

    ax.set_xlim([0, BOX_SIZE[0]])
    ax.set_ylim([0, BOX_SIZE[1]])
    ax.set_zlim([0, BOX_SIZE[2]])
    ax.set_xlabel("X")
    ax.set_ylabel("Y")
    ax.set_zlabel("Z")
    ax.set_title("3D Packing Visualization")
    plt.tight_layout()
    plt.show()


## 🚀 最適化の実行（FMQA）

In [ ]:

client = FixstarsClient()
client.token = "your_token"  # 🔒 Fixstars Amplify APIトークンをここに記入
client.parameters.timeout = 5000

sampler = FMQASampler(client)
variables = [BinarySymbol(NUM_CANDIDATES) for _ in range(NUM_CARTONS)]

best_result = sampler.sample(evaluate_solution, variables, num_search=100)

print("Best score:", best_result.score)
print("Best assignment:", best_result.values)
visualize_packing(best_result.values)
